# Predicting street dust concentrations with dilated causal convolutional neural networks

## Problem and data description

A model for predicting street dust concentrations for 48 hours ahead is developed in this project work. Model architecture has got inspiration from adapted WaveNet network by *Borovykh et al., 2018* . Network contains stacks of dilated convolutions that allow it to access a broad range of history when forecasting, residual connections, a SELU activation function and conditioning with multivariate input data. PyTorch is used to build the model.

Street dust is a recurring problem in Finland especially in springtime because sand and other road surface material are released into the air when snow and ice have melted and streets have dried. Street dust consist of materials from street surfaces, sanding materials, and materials from brakes and studded tires. Street dust concentrations are typically higher also during end of the year when road sanding usually begins, but roads are not yet fully covered by snow and ice. Construction sites and street work also cause street dust. Most street dust particles are less than 10 micrometres (PM10) in diameter. 

Street dust can cause respiratory tract irritation causing cold-like symptoms, coughing and irritation to the eyes, nose and throat. People suffering from asthma, small children and and those suffering from the coronary artery disease are most vulnerable to adversarial effects of street dust. Weather conditions,  quality of the road surface and amount of traffic affects street dust concentration levels. Wet conditions prevent dust from street surfaces rising into the air, and gusty wind can lift street dust efficiently from unclean roads. The dustiest areas are busy streets and cross-roads during rush hours.

In [ ]:
import os
import time
import copy
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_load

Input data is history observations of hourly PM10 concentrations, amount of rain, temperature and wind speed. Concentration data is from air quality monitoring station located at near busy street (Mäkelänkatu) in Helsinki, and weather data is from nearby weather station (Kumpula). Data is from period 01.01.2017-30.4.2019, i.e. 28 months of hourly observations. Data is downloaded from https://en.ilmatieteenlaitos.fi/download-observations#!/.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load data
df = pd.read_csv('/content/drive/My Drive/data/PM10_weather_data_LONG2.csv',  parse_dates=[['Year', 'Month', 'Day', 'Time']])
df.tail()

In [ ]:
# print length of data and start and end time
data_size = df.shape[0]
print("Total amount of hours:", data_size)
first = df['Year_Month_Day_Time'].iloc[0]
last = df['Year_Month_Day_Time'].iloc[-1]
print("Data ranges from %s to %s." % (first, last))

PM10 concentration values varies between -5 and 280 micrograms/m^3. Negative values are probably due to some malfunction in measurement device. From the histograms at Figure 1 we can see that distributions of PM10 concentration and amount of rain are skewed. Distribution of temperature and wind speed are more closer to normal distribution. Time series of PM10 concentration and temperature are shown in Figure 2. 

In [ ]:
# plot histograms of input data
plt.rcParams['figure.figsize']=(18,6)
fig3, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
ax1.hist(df_fill['PM10 concentration (ug/m3)'], 50)
ax2.hist(df_fill['Amount of rain (mm/h)'], 10)
ax3.hist(df_fill['Temperature (degC)'], 40)
ax4.hist(df_fill['Wind speed (m/s)'], 40)
ax1.set_xlabel('PM10 concentration (ug/m3)')
ax2.set_xlabel('Amount of rain (mm/h)')
ax3.set_xlabel('Temperature (degC)')
ax4.set_xlabel('Wind speed (m/s)')
fig3.savefig('hist.png')

<img src="hist.png" width=1000 height=600 />

Figure 1. Histograms of input data.

In [ ]:
# plot temperature and PM10 concentration
plt.rcParams['figure.figsize']=(18,8)
fig2, ax1 = plt.subplots()
ax2 = ax1.twinx() 
ax1.plot(df_fill['Temperature (degC)'], color='orange', label='Temperature')
ax2.plot(df_fill['PM10 concentration (ug/m3)'], label='PM10 concentration')
ax1.set_ylabel('Temperature (degC)')
ax2.set_ylabel('PM10 concentration (ug/m3)')
ax2.set_ylim(0, 250)
plt.gcf().autofmt_xdate()
plt.margins(x=0,y=0)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax2.set_title('PM10 concentration and temperature')
fig2.savefig('temp_plot.png')

<img src="temp_plot.png" width=1000 height=600 />

Figure 2. Time series of PM10 concentration and temperature.

In [ ]:
# plot wind speed and PM10 concentration
plt.rcParams['figure.figsize']=(18,8)
fig2, ax1 = plt.subplots()
ax2 = ax1.twinx() 
ax1.plot(df_fill['Wind speed (m/s)'], color='orange', label='Wind speed')
ax2.plot(df_fill['PM10 concentration (ug/m3)'], label='PM10 concentration')
ax1.set_ylabel('Wind speed (m/s)')
ax2.set_ylabel('PM10 concentration (ug/m3)')
ax2.set_ylim(0, 250)
plt.gcf().autofmt_xdate()
plt.margins(x=0,y=0)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax2.set_title('PM10 concentration and wind speed')

In [ ]:
# plot amount of rain and PM10 concentration
plt.rcParams['figure.figsize']=(18,8)
fig1, ax1 = plt.subplots()
ax2 = ax1.twinx() 
ax1.bar(df_fill['Amount of rain (mm/h)'].index, df_fill['Amount of rain (mm/h)'], alpha=0.7, color='orange', label='Amount of rain')
ax2.plot(df_fill['PM10 concentration (ug/m3)'], label='PM10 concentration')
ax1.set_ylabel('Amount of rain (mm/h)')
ax2.set_ylabel('PM10 concentration (ug/m3)')
ax2.set_ylim(0, 250)
plt.gcf().autofmt_xdate()
plt.margins(x=0,y=0)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax2.set_title('PM10 concentration and amount of rain')

## Method

The adopted dilated causal convolutional neural network stucture is based on Google's famous WaveNet model from first developed for audio forecasting (*van den Oord et al., 2016*).  The intuition behind using convolutional neural networks (CNN) for time series forecasting is that CNNs should be able to learn filters that represent certain repeating patterns in the time series, similarly as with images,  and use these to forecast the future values. With timeseries we use one dimensional convolution. Training a model on multivariate input data allows the network to exploit the correlation structure between these multiple time series and learning long-term temporal dependencies in between series . The advantage of the CNN over the recurrent-type network is that due to the convolutional structure of the network, the number of trainable weights is small, resulting in a much more efficient training and predicting *(Borovykh et al., 2018)*.

In a dilated convolution the filter is applied to every "dilation rate"th element in the input vector, allowing the model to efficiently learn connections between far-apart data points. Using the dilated convolutions instead of regular ones allows the output to be influenced by more nodes in the input i.e. expand the receptive field of the model.  The receptive field of the model is the number of neurons (time steps) in the input that can modify the output in the final layer, i.e. the forecasted time series. The receptive field depends on the number of layers L and the filter size k. In this model architecture the dilations are increasing in every layer by a factor of two: 1, 2, 4, 8, etc. The filter size (same as kernel size) is chosen to be 2 and number of layers is 9, so the receptive field of the model is 512 hours (3 weeks) in this experiment. Figure 3 represents illustration of dilated layers.

<img src="dilate.png" width=1000 height=600 />

Figure 3. Illustration of dilation layers (from https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Conv_Full.ipynb).

The word causal indicates that the convolution output should not depend on future inputs. In time series this is equivalent to padding the input (in the left) with a vector of zeros of the size of the receptive field. Conditioning with multiple input time series is done with separate convolutions with SELU (scaled exponential linear unit) activation function and skip connections for each input series and then concatenated together to form output from the first layer. In the first layer dilation rate is 1. The skip connections are parametrized by 1×1 convolutions, i.e. kernel size is 1, and number of output channels (filters) of 1. If some condition does not improve the forecast, the model can simply learn to discard this condition by setting the weights in the parametrized skip connection to zero. In the first layer each input time series skip connections and convolution outputs are concatenated together and then they are passed to 1×1 convolution to form the output of first layer. Subsequent layers have convolution with increasing dilation rate and SELU activation function and residual connection from input to output of the layer. Residual connection is added to the convolution output to form the final output of the layer. After the last layer there is final 1×1 convolution which outputs the forecasted time series. Only last 48 hours are taken from the final output.

Figure 4 illustrates the model architecture of Borovykh et al., 2018. In our adopted version separate skip connections and convolution outputs from the input time series are concatenated not summed. Both methods are commonly used in adding residual/skip connections to the convolution output, summing and concatenating. We decided to use latter method in hope that the information from conditioning input time series would propagate better in the network. We have also extra 1×1 convolution in the first layer before the output to decrease the number of filters after concatenation operation. We tried also with different activation functions, namely ReLU and LeakyReLU, the architecture presented here is with SELU activations. SELU was chosen in first place because it can handle also negative values. We didn't notice clear difference in performance with these different activation functions. Number of filters were chosen to be quite small in the convolutional layers to avoid overfitting with our quite small dataset. Hidden size of convolutional layers was set to 3. We tried also bigger values but those yielded overfitting.

<img src="Augmented_wavenet.png" width=1000 height=600 />

Figure 4. Model architecture of augmented Wavenet by Borovykh et al., 2018 (figure taken from the article).

Below is the model structure which is used in this project.

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, hidden_size, kernel_size, dilation_rate, bias=False):
        super(ConvBlock, self).__init__()
        self.hidden_size = hidden_size,
        self.kernel_size = kernel_size,
        self.dilation_rate = dilation_rate
          
        self.conv = nn.Conv1d(hidden_size, hidden_size, kernel_size=kernel_size, dilation=dilation_rate)
        self.relu = nn.SELU()


    def forward(self, inputs):
        residual = inputs
        pads = (self.kernel_size[0] - 1) * self.dilation_rate
        inputs_padded = F.pad(inputs, (pads, 0))
        
        layer_out = self.relu(self.conv(inputs_padded))
        network_out = torch.add(residual, layer_out)
        return network_out


In [ ]:
class DilatedConvNet(nn.Module):
    def __init__(self, n_layers, num_inputs, hidden_size, kernel_size, dilation_rate=1, bias=False):
        super(DilatedConvNet, self).__init__()
        self.n_layers = n_layers,
        self.hidden_size = hidden_size,
        self.kernel_size = kernel_size,
        self.dilation_rate = dilation_rate
        self.num_inputs = num_inputs
        
        # first layer
        self.conv1 = nn.Conv1d(1, hidden_size, kernel_size=kernel_size, dilation=dilation_rate)
        self.relu1 = nn.SELU()
        self.skip_first = nn.Conv1d(1, 1, kernel_size=1, dilation=dilation_rate)
        # decrease the number of filters to hidden size
        self.conv2 = nn.Conv1d((hidden_size+1)*num_inputs, hidden_size, kernel_size=1, dilation=dilation_rate)
        
        # other layers
        other_layers = [ConvBlock(hidden_size=hidden_size, kernel_size=kernel_size, dilation_rate=2 ** i) for i in range(1, n_layers)]
        self.group = nn.Sequential(*other_layers)
        
        # last layer
        self.dense = nn.Conv1d(hidden_size, 1, kernel_size=1, dilation=dilation_rate)
        
        # initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                n = m.kernel_size[0] * m.out_channels
                m.weight.data.normal_(0, np.sqrt(2. / n))
                

    def forward(self, inputs):
        # first layer
        # padding the input with a vector of zeros of the size of the receptive field
        pads = (self.kernel_size[0] - 1) * self.dilation_rate
        inputs_padded = F.pad(inputs, (pads, 0))
        
        input_layer = []
        for i in range(self.num_inputs):
          layer_out = self.relu1(self.conv1(inputs_padded[:,i,:].view(inputs_padded.shape[0],1,-1)))
          skip_out = self.skip_first(inputs[:,i,:].view(inputs.shape[0],1,-1))
          out = torch.cat((skip_out, layer_out), dim=1)
          input_layer.append(out)
        out = torch.cat(input_layer, dim=1)
        out = self.conv2(out)
        
        # other layers
        outputs = self.group(out)
        
        # last layer
        network_out = self.dense(outputs)
        outputs = network_out[:,:,-output_steps:] # output only last 48 hours
        return outputs

Convolutional layer weights are initialized with zero-mean Gaussian distribution and standard deviation of sqrt(2/z), where z is the total number of trainable parameters in the layer. Objective function was chosen to be mean absolute error and we used Adam gradient descent with learning rate 0.001. L2 regularization, i.e. weight decay, was used to avoid overfitting with regularization rate 0.001. These parameters was chosen to be same asi in (Borovykh et al., 2018). Hyperparameter search was not done.

In the chosen model architecture the output is a vector of 48 values, i.e. the model predicts directly concentrations for the next 48 hours. The model is static, it is trained only one time. Other option could have been to implement architecture which have 48 separate models, each predicting value for certain hour. Or model wich will output only next hour, then updated with new input data, and this woul be looped over 48 iterations.

In [ ]:
# model parameters
n_layers = 9
hidden_size = 3
kernel_size = 2
num_inputs = 4 # number of input features (number of input time series)
model = DilatedConvNet(n_layers=n_layers, num_inputs=num_inputs, hidden_size=hidden_size, kernel_size=kernel_size)
print(model.to(device))
model.to(device)

# using mean absolute error as a criterion
criterion = nn.L1Loss()
# using Adam optimizer and L2 regularization
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
# learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Feed a batch of data from the training data to test the network
with torch.no_grad():
    dataiter = iter(trainloader)
    values, labels = dataiter.next()
    values = values.to(device)
    print('Shape of the input tensor:', values.shape)

    y = model(values)
    print(y.shape)

## Data preparation

There are some missing values in the data,  1% of PM10 concentration observations and less than  1% of weather observations are missing. Missing time stamps are filled using linear interpolation. The amount of missing data is small and because the causal nature of input data, the missing time stamps are decided to fill, not delete. Input data consist of 20 400 hours which is divided to training (18 000 hours) and test (2 400 hours) sets. Training data is separated to equal length sequences of training input (512 hours) and training target (48 hours) data in overlapping moving window format. Target data is non-overlapping between sequences. When the training data is splitted using overlapping moving window method there will be  35 separate sequences, i.e. samples. These are further divided to training (30 samples) and validation (5 samples ) sets. 

In [ ]:
# check missing values
print("Missing values:\n")
print(df.isnull().sum(), "\n")
# put the time as index
df = df.set_index('Year_Month_Day_Time')

In [ ]:
# check min and max values for the concentration i.e. predictant
print(df['PM10 concentration (ug/m3)'].min())
print(df['PM10 concentration (ug/m3)'].max())

In [ ]:
# use linear interpolation for filling the missing values
df_fill = df.interpolate()
print(df_fill.isnull().sum(), "\n")
print(df_fill.shape[0])

In [ ]:
# check data types
df_fill.dtypes

In [ ]:
# function to split dataset to training and test sets
def split_dataset(data, split_point):
    train, test = data[0:split_point], data[split_point:data.shape[0]]
    return train, test

In [ ]:
# split input data to train and test batches
input_data = np.stack((df_fill['PM10 concentration (ug/m3)'].values, df_fill['Amount of rain (mm/h)'].values, df_fill['Temperature (degC)'].values, df_fill['Wind speed (m/s)'].values), axis=-1)
print(input_data.shape)
train, test = split_dataset(input_data, 18000)

In [ ]:
# check the shapes of train and test sets
# and check that last value of train data and first value of test data are different
print(train.shape)
print(test.shape)
print(train[-1,0])
print(test[0,0])

Because the scales are different in input data, it is standardized by removing the mean and scaling to unit variance. There are lot's of zeros in one of the input time series (amount of rain), so also for this reason the scaling is needed. The scaling is done separately for each split, so that the information from training targets doesn't leak to training inputs. When the trained model is used for prediction the scaling have to be done also for the input data, and of course re-scaling before inputting the predicted values.

In [ ]:
# convert training data into batches of equal length sequences of train_inputs and train_targets
# option for scaling the data is included
# modified function, original from https://machinelearningmastery.com/how-to-develop-convolutional-neural-networks-for-multi-step-time-series-forecasting/
def split_equal_sequences(data, n_input, n_out, scaling=False):
    X, y = list(), list()
    in_start = 0
    # step over the training data one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end < len(data):
            x_input = data[in_start:in_end, :]
            y_input = data[in_end:out_end, 0].reshape(n_out,1)
        # standardize data by removing the mean and scaling to unit variance
        if scaling:
            scaler = StandardScaler().fit(x_input)
            scaler2 = StandardScaler().fit(x_input[:,0].reshape(-1,1))
            x_input = scaler.transform(x_input)
            y_input = scaler2.transform(y_input.reshape(-1,1))
      
        X.append(x_input)
        y.append(y_input)
		# move along one time step
        #in_start += n_input
        in_start += 1 #try with overlapping moving window
    return np.array(X), np.array(y)

In [ ]:
# Length of input data (hours of history data) and lenght of prediction period
input_steps = 512 # 3 weeka
output_steps = 48

In [ ]:
# split training data to input and target sequences. 
train_inputs, train_targets = split_equal_sequences(train, input_steps, output_steps, scaling=True)
print(train_targets.shape)
print(train_inputs.shape)

# reshape data to input format for convolution layer [batch, channels, steps] and tranform to tensors.
train_inputs = train_inputs.reshape(train_inputs.shape[0], train_inputs.shape[2], -1)
train_targets = train_targets.reshape(train_targets.shape[0], train_targets.shape[2], -1)
x = torch.tensor(train_inputs, device=device, dtype=torch.float)
y = torch.tensor(train_targets, device=device, dtype=torch.float)
print(x.size())
print(y.size())
# combine tensors to dataset
dataset = torch.utils.data.TensorDataset(x, y)

In [ ]:
# divide the training dataset further to training and validation datasets.
idx = list(range(len(dataset)))
train_idx = idx[:15000]
val_idx = idx[15000:]
dataset_train = torch.utils.data.Subset(dataset, train_idx)
dataset_valid = torch.utils.data.Subset(dataset, val_idx)
print(len(dataset_train))
print(len(dataset_valid))

## Experiments and results

Model is run in Google colab with GPUs. Different setups were tested with different numbers of layers, kernel size, hidden size and input time steps. Adding a dropout layer at the end of each convolutional layer was also tested. None of these tested versions was remarkably better than other.

In [ ]:
device = torch.device('cuda:0')
#device = torch.device("cpu")

In [ ]:
# load training and validation datasets to torch Dataloader format
trainloader = torch.utils.data.DataLoader(dataset_train, batch_size=15, shuffle=True, pin_memory=False)
validloader = torch.utils.data.DataLoader(dataset_valid, batch_size=5, shuffle=True, pin_memory=False)

In [ ]:
# train model and monitor training and validation error
# function modified from Pytorch tutorial: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#model-training-and-validation-code
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    since = time.time()
    val_loss_history = []
    train_loss_history = []

    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data
            for inputs, targets in dataloaders[phase]:
                inputs = inputs.to(device)
                targets = targets.to(device)
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                  loss = criterion(outputs, targets)
                  # backward + optimize only if in training phase
                  if phase == 'train':
                    loss.backward()
                    optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            
            if (epoch % 10) == 0:
              print('Epoch {}/{} {} loss: {:.4f}'.format(epoch, num_epochs - 1, phase, epoch_loss))
              #print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            if phase == 'val':
                val_loss_history.append(epoch_loss)
            if phase == 'train':
                train_loss_history.append(epoch_loss)
            
            scheduler.step(epoch_loss)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return model, train_loss_history, val_loss_history

In [ ]:
# put training and validation sets to one dictionary for the use of training function
dataloaders_dict = {'train' : trainloader, 'val': validloader}

In [ ]:
# train model and gather training and validation losses to list
model_trained, train_loss_history, val_loss_history = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=100)

In [ ]:
# save model
model_filename = 'cnn_model.pth'
torch.save(model_trained.state_dict(), model_filename)

Training and validation losses are shown in Figure 5. Model underfits slightly. We tried training also with more epochs (5 000 epochs), but the validation loss started to increase slowly. We decided to cut the training in 500 epochs. For the underfitting problem solution could be to increase the input data or change the model architecture.

In [ ]:
# plot training and validation error
plt.figure(figsize=(10,6))
plt.plot(np.arange(len(train_loss_history)), train_loss_history, label='train')
plt.plot(np.arange(len(val_loss_history)), val_loss_history, label='val')
plt.ylim([0,2])
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error Loss')
plt.title('Loss Over Time')
plt.legend()
plt.savefig('loss.png')

<img src="loss.png" width=800 height=600 />

Figure 5. Training and validation losses.

For the purpose of prediction the test set was divided to equal lenght batches of 48 hours. There was 49 test batches. In the prediction phase input data was fed to model in iterative way. In first iteration last *input_steps* from input data (these hours weren't part of training data, they belonged to validation data) is fed to model and prediction is compared to the first 48 hours of test data. In the next iteration first 48 hours from test data is appended to input data and the prediction is done with this new input data. Mean absolute error (MAE) score is calculated to the prediction. MAE scores for each test batch are shown in Figure 6. It can bee seen that mean absolute error gets bigger when moving away from the split point of training data. In the first 10 batches MAE is less than 10, but after that is increases substantially. This is expectable because model isn't updated between iterations with new input data. The average MAE for all the test batches is 14.03 and for the first 10 batches 5.53.

In the Figure 7 there are shown plots of input data, true target data and predictions for the first test batches. It's obvious that the model performace is not very good. The model doesn't capture the high concentration peaks, neither the shape of the variation. We tried also with bigger number of filters but that resulted even worse predictions with huge variation. Model predictions are compared to naive forecast, i.e. last 48 hours are used as a  prediction for next 48 hours. The MAE score of naive prediction is 17.86. Model outperfoms the naive forecast.

In [ ]:
model_trained.load_state_dict(torch.load(model_filename, map_location=lambda storage, loc: storage))
model_trained.to(torch.device("cpu"))
print('Model loaded from %s' % model_filename)

In [ ]:
# print weights of second layers convolution
model_trained.group[0].conv.weight

In [ ]:
model_trained.conv2.weight

In [ ]:
# function for predicting next 48 hours values based on "n_input" previous time steps using the trained model
def predict(model, input_data, n_input, device):
    # retrieve last observations for input data
    # last observations from input data have not been used in training
    inputs = input_data[-n_input:, :]
    # Standardize input data
    scaler = StandardScaler().fit(inputs)
    scaler2 = StandardScaler().fit(inputs[:,0].reshape(-1,1))
    inputs = scaler.transform(inputs)
    # reshape into [1, channels, n_input]
    inputs = inputs.reshape(1, inputs.shape[1], inputs.shape[0])
    inputs = torch.tensor(inputs, device=device, dtype=torch.float)
    # forecast the next 48 hours
    model.eval()
    model.to(device)
    with torch.no_grad():
        inputs = inputs.to(device)
        outputs = model.forward(inputs)
        outputs = outputs.cpu().data.numpy()
        # scale back to original form
        outputs_rescaled = scaler2.inverse_transform(outputs.reshape(outputs.shape[2],outputs.shape[1]))
    return np.squeeze(outputs_rescaled)

In [ ]:
# function for splitting test data to non-overlapping batches of 48 hours
def split_test_data(data, n_out):
    y = list()
    in_start = 0
    for _ in range(len(data)):
      # define the end of the sequence
      in_end = in_start + n_out
      # check that we have enough data for this instance
      if in_end < len(data):
        batch = data[in_start:in_end]
        y.append(batch)
        in_start += n_out
    return np.array(y)

In [ ]:
# split test data to batches of 48 hours
test_batches = split_test_data(test, output_steps)
print(test_batches.shape)

In [ ]:
def evaluate_predictions(actual, predicted):
    scores = list()
    # calculate mean absolute error (MAE) score for each 48 hour prediction period
    for i in range(actual.shape[0]):
        # calculate MAE
        mae = mean_absolute_error(actual[i, :], predicted[i, :])
        # store
        scores.append(mae)
        # calculate average MAE
        mae_average = sum(scores) / len(scores)
        return mae_average, scores

In [ ]:
# evaluate model predictions using walk-forward validation
# test data is divided to batches of 48 hours and for each batch MAE value is calculated
def evaluate_model(model, input_data, test_data, n_input, device):
    # walk-forward validation over each 48h batch
    predictions = list()
    for i in range(len(test_data)):
        # predict the 48 hours
        yhat = predict(model, input_data, n_input, device)
        # store the predictions
        predictions.append(yhat)
        # get real observation from the test data and add to input_data for predicting the next 48 hours
        input_data = np.concatenate([input_data, test_data[i, :, :]])
    # evaluate predictions for each 48 hours batch
    predictions = np.array(predictions)
    mae_average, scores = evaluate_predictions(test_data[:,:,0], predictions)
    return mae_average, scores, predictions

In [ ]:
# calculate MAE values for the mode predictions
mae_average, mae_batches, predictions = evaluate_model(model_trained, train, test_batches, input_steps, device)
print(mae_average)
# MAE for the first 10 batches
mae_average_10batch = sum(mae_batches[:10]) / len(mae_batches[:10])
print(mae_average_10batch)

In [ ]:
# plot the MAE over different test batches
plt.figure(figsize=(10,6))
plt.plot(range(len(mae_batches)), mae_batches)
plt.title('MAE over test batches')
plt.legend(['MAE'])
plt.xlabel('Batch number')
plt.ylabel('Mean absolute error')
plt.savefig('MAE_batches.png')

In [ ]:
# function for plotting predictions and true observations to same figure
# figure is created per test batch
# modified function, original from https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Conv_Full.ipynb
def plot_predictions(input_data, target_data, predictions, batch_ind, train_tail_len):
    input_series = input_data[:,0]
    pred_series = predictions[batch_ind,:]
    target_series = target_data[batch_ind,:,0]
      
    input_series_tail = np.concatenate((input_series[-train_tail_len:],target_series[:1]), axis=0) 
    x = input_series_tail.shape[0]
    pred_steps = pred_series.shape[0]
      
    plt.figure(figsize=(10,6))   
      
    plt.plot(range(1,x+1),input_series_tail)
    plt.plot(range(x,x+pred_steps),target_series,color='orange')
    plt.plot(range(x,x+pred_steps),pred_series,color='teal',linestyle='--')
      
    plt.title('Batch index %d' % batch_ind)
    plt.legend(['Input data','Target series','Predictions'])
    plt.xlabel('Hours')
    plt.ylabel('PM10 concentrations [micro g/m³]')

In [ ]:
plot_predictions(train, test_batches, predictions, batch_ind=0, train_tail_len=72)
plt.savefig('forecast0.png')

In [ ]:
plot_predictions(train, test_batches, predictions, batch_ind=1, train_tail_len=72)
plt.savefig('forecast1.png')

In [ ]:
plot_predictions(train, test_batches, predictions, batch_ind=15, train_tail_len=72)
plt.savefig('forecast2.png')

In [ ]:
# make naive forecast by taking last 48 hours as the prediction for next 48 hours and calculate MAE
def evaluate_naive(input_data, test, n_input):
    predictions = list()
    for i in range(len(test)):
        # take last 48 hours from training data
        value = input_data[-n_input:, 0]
        predictions.append(value)
        # get real observation from test data and add to input_data
        input_data = np.concatenate((value, test[i, :, 0]), axis=0).reshape(-1,1)
    # evaluate predictions for each test batches
    predictions = np.array(predictions)
    mae_average, scores = evaluate_predictions(test[:,:,0], predictions)
    return mae_average, scores

In [ ]:
# calculate MAE values for the naive forecast
naive_mae_average, naive_mae_batches = evaluate_naive(train, test_batches, output_steps)
print(naive_mae_average)

## 4. Conclusions

We developed a model for predicting street dust concentrations for 48 hours ahead using past 3 weeks observations. Model is implemented with Pytorch. Model architecture contains stacks of dilated convolutions, residual connections, a SELU activation function and conditioning with multivariate input data.  Input data is hourly observations of PM10 concentrations, amount of rain, temperature and wind speed of 28 months. Input data divided into training, validation and test sets with moving window method, and scaled to zero mean and unit variance before feeding to model. Model outputs directly the 48 hours forecast, and the same model is used for making the predictions for all the test batches.

The average MAE of the model for all the test batches is 14.03 and for the first 10 batches 5.53.The MAE score of naive prediction is 17.86. Model outperfoms the naive forecast, but the prediction results are not good. The model doesn't capture the high concentration peaks, neither the overall shape of the concentration curve. The training and validation losses indicates that the model is underfitting slightly. Adding more layers to the model, i.e. expanding the receptive field, could increase the model performance.  Also adding hourly traffic amount as a conditioning data could affect positively to the model performance.

## References


A. Borovykh, S. Bohte, and C. W. Oosterlee, Conditional Time Series Forecasting with Convolutional Neural Networks, ArXiv e-prints, (2018)

A. van den Oord, S. Dieleman, H. Zen, K. Simonyan, O. Vinyals, A. Graves, N. Kalchbrenner, A. Senior, and K. Kavukcuoglu, WaveNet: A Generative Model for Raw Audio, ArXiv e-prints, (2016)